In [1]:
# Use Structured Streaming 

In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.master('local')
.appName('simpleSpark')
.config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
.getOrCreate())

sc = spark.sparkContext

In [4]:
# Read the message from Kafka stream

In [5]:
df = spark \
.readStream \
.format("kafka") \
.option('kafka.bootstrap.servers','kafka:9092') \
.option('subscribe','ingestion-topic') \
.load()

# df.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")

In [8]:
# Create a small temporary view for SparkSQL
df.createOrReplaceTempView("message")


In [ ]:

# write out the message
res = spark.sql('select * from message')

res.writeStream.format('console').outputMode('append').start()

In [ ]:
# Write the message back into Kafka in another topic that you are going to listen to with a local consumer
ds = df \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("topic", "spark-output") \
  .option("checkpointLocation", "/tmp") \
  .start() \
  .awaitTermination()